Generates Voronoi Figures (don't have to run, I already uploaded the output to this part above, just go to the second code chunk)

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import os

src = r"C:\Users\ctr22\Downloads\gadm41_PSE_1.json\gadm41_PSE_1.json"  # your path

gdf = gpd.read_file(src)
print("CRS:", gdf.crs)
print("Number of features:", len(gdf))
print("\nColumns:\n", list(gdf.columns))
print("\nFirst rows:")
display(gdf.head())


# Try common name columns and show unique values (limit)
candidates = ["NAME_0","NAME_1","NAME","GID_0","GID_1","VARNAME_1","ENGTYPE_1"]
found = {}
for c in candidates:
    if c in gdf.columns:
        vals = gdf[c].unique()
        found[c] = vals[:10] if len(vals)>10 else vals

print("\nDetected candidate name columns and sample values:")
for k,v in found.items():
    print(k, "->", v)

# Strategy to pick Gaza: try NAME_1 or similar containing 'Gaza' (case-insensitive),
# otherwise pick polygon(s) whose centroid latitude < 31.7 (southern strip).
gaza_gdf = None

# 1) look for textual match in candidate columns
for col in ["NAME_1","NAME","VARNAME_1","NAME_0","ENGTYPE_1"]:
    if col in gdf.columns:
        mask = gdf[col].astype(str).str.contains("gaza", case=False, na=False)
        if mask.any():
            gaza_gdf = gdf[mask].copy()
            print(f"Selected Gaza from column '{col}' (text match).")
            break

# 2) if not found, try numeric admin code if available (common GADM codes)
if gaza_gdf is None:
    for col in ["GID_1","GID_0"]:
        if col in gdf.columns:
            # print a sample to help you manually choose
            print(f"Column {col} exists. Sample values: {gdf[col].unique()[:20]}")
            # don't auto-pick by code, leave for manual if needed

# 3) fallback: geometry centroid latitude filter (safe for Gaza)
if gaza_gdf is None:
    # compute centroids (ensure geometry valid)
    gdf = gdf[~gdf.geometry.is_empty].copy()
    centroids = gdf.geometry.centroid
    lat_vals = centroids.y
    # Gaza is southern and coastal; centroid latitude < 31.7 tends to select Gaza features
    mask = lat_vals < 31.7
    if mask.any():
        gaza_gdf = gdf[mask].copy()
        print("Selected Gaza by centroid latitude < 31.7 (fallback).")
    else:
        print("No features matched centroid latitude < 31.7; trying latitude < 31.9 ...")
        mask = lat_vals < 31.9
        if mask.any():
            gaza_gdf = gdf[mask].copy()
            print("Selected Gaza by centroid latitude < 31.9 (fallback).")

# If still None, show hint and exit
if gaza_gdf is None or gaza_gdf.empty:
    raise RuntimeError("Could not automatically identify Gaza polygon. Inspect gdf.columns and gdf.head() for clues.")

# Dissolve into a single polygon (in case Gaza is multiple features)
gaza_union = gaza_gdf.dissolve(by=None).reset_index(drop=True)
print("Resulting Gaza features:", len(gaza_union))
# Optional: quick plot to verify
ax = gaza_union.plot(figsize=(6,6))
ax.set_title("Extracted Gaza polygon (verify visually)")
plt.show()

# Save to GeoJSON
out_path = r"C:\Users\ctr22\Downloads\gaza_boundary.geojson"
gaza_union.to_file(out_path, driver="GeoJSON")
print("Saved Gaza boundary to:", out_path)


Generates maps

In [1]:
"""
gaza_catchment_safe.py

Safer, robust end-to-end pipeline to build Gaza-clipped hospital catchment (Voronoi) maps
and overlay ACLED attack heatmaps for specified date windows.

Key improvements vs earlier script:
 - Uses shapely.ops.voronoi_diagram when available (fast, bounded via envelope)
 - SciPy fallback builds Voronoi inside a bounded bbox (no giant buffers)
 - NumPy 2.0 compatible (uses np.ptp)
 - Safety caps and timing prints to avoid infinite/very long runs
 - Robust normalization of Gaza boundary inputs
 - Assigns leftover slivers inside Gaza to nearest hospital to ensure full partitioning

Edit the HOSP_PATH, ACLED_PATH, and GAZA_SHAPEFILE variables at the top before running.
"""

import os
import time
from datetime import datetime, timedelta
import itertools
import random

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, box, shape
from shapely.ops import unary_union
from scipy.spatial import Voronoi
import folium
from folium.plugins import HeatMap
import warnings
warnings.filterwarnings("ignore")

# ---------------- USER CONFIG ----------------
HOSP_PATH = "Hospitals_OpenCloseoverTime.xlsx"
ACLED_PATH = "ACLED_May_09_25_Gaza.xlsx"
GAZA_SHAPEFILE = "gaza_boundary.geojson"  # set to None to use bbox fallback

OUT_DIR = "."
os.makedirs(OUT_DIR, exist_ok=True)

# Date windows requested
WINDOWS = [
    (datetime(2023, 12, 1), datetime(2024, 4, 30)),
    (datetime(2024, 11, 1), datetime(2025, 1, 30)),
]

CRS_WGS84 = "EPSG:4326"
CRS_WEBMERC = "EPSG:3857"

# Safety / performance tuning
VORONOI_POINT_CAP = 400          # abort if more hospitals than this
VORONOI_BUFFER_METERS = 15000    # buffer for bounding box in projected coords (meters)
BUFFER_DEG = 0.05

# ---------------- Helpers: I/O & schedule parsing ----------------

def read_hospitals_table(path):
    """
    Parse the Hospitals_OpenCloseoverTime.xlsx file into:
      - hospitals_df: columns ['Hospital','lon','lat', plus raw schedule marker columns]
      - schedule_meta: list of (col_name, 'Open'/'Closed', datetime) for columns that carry dates
    This function is robust to the header+date-in-row layout described earlier.
    """
    raw_head = pd.read_excel(path, header=None, nrows=2)
    full = pd.read_excel(path, header=0)
    cols = list(full.columns)

    def find_col(opts):
        for o in opts:
            for c in cols:
                if isinstance(c, str) and c.strip().lower().startswith(o.lower()):
                    return c
        return None

    hosp_col = find_col(["hospital", "name"])
    lon_col = find_col(["longitude (x)", "longitude", "lon", "x"])
    lat_col = find_col(["latitude (y)", "latitude", "lat", "y"])
    if hosp_col is None or lon_col is None or lat_col is None:
        raise ValueError(f"Couldn't detect Hospital/lon/lat columns. Found: {cols}")

    # detect Open/Closed header columns and try to read date from second row
    schedule_meta = []
    for c in cols:
        if isinstance(c, str):
            lc = c.strip().lower()
            if lc.startswith("open") or lc.startswith("closed"):
                typ = "Open" if lc.startswith("open") else "Closed"
                dt = None
                try:
                    idx = cols.index(c)
                    raw = raw_head.iat[1, idx]
                    if pd.notnull(raw):
                        dt = pd.to_datetime(raw)
                except Exception:
                    dt = None
                schedule_meta.append((c, typ, dt))

    # keep only entries which have a date (we need timestamps)
    schedule_meta = [(c,t,d) for (c,t,d) in schedule_meta if d is not None]

    # fallback: detect date-like column headers (rare case)
    if not schedule_meta:
        for c in cols:
            try:
                d = pd.to_datetime(c)
                # try to infer whether this header corresponds to Open/Closed by reading raw_head[0, idx]
                typ = None
                try:
                    idx = cols.index(c)
                    hdr0 = raw_head.iat[0, idx]
                    if isinstance(hdr0, str) and "open" in hdr0.lower():
                        typ = "Open"
                    elif isinstance(hdr0, str) and "closed" in hdr0.lower():
                        typ = "Closed"
                except Exception:
                    typ = "Open"
                schedule_meta.append((c, typ, d))
            except Exception:
                continue

    hospitals_df = full[[hosp_col, lon_col, lat_col]].rename(columns={hosp_col: "Hospital", lon_col: "lon", lat_col: "lat"})
    hospitals_df["lon"] = pd.to_numeric(hospitals_df["lon"], errors="coerce")
    hospitals_df["lat"] = pd.to_numeric(hospitals_df["lat"], errors="coerce")
    # copy any schedule marker columns we detected
    for c,_,_ in schedule_meta:
        if c in full.columns:
            hospitals_df[c] = full[c]

    return hospitals_df, schedule_meta

def build_hospital_open_intervals(hospitals_df, schedule_meta):
    """
    From schedule_meta (col, typ, date), produce per-hospital ordered list of (date, status).
    If per-row cells contain markers under schedule columns we respect them; otherwise use global schedule.
    """
    # prepare events sorted by date
    events = sorted([(pd.to_datetime(d).to_pydatetime(), col, typ) for (col, typ, d) in schedule_meta], key=lambda x: x[0])
    hospital_intervals = {}
    for _, row in hospitals_df.iterrows():
        name = row["Hospital"]
        changes = []
        # If per-row markers exist, use them
        for dt, col, typ in events:
            col_name = col
            val = row.get(col_name, None) if col_name in row.index else None
            if pd.notnull(val) and str(val).strip() != "":
                changes.append((dt, typ))
        # If none, fall back to global events
        if not changes:
            for dt, col, typ in events:
                changes.append((dt, typ))
        # compress consecutive duplicates
        changes_sorted = sorted(changes, key=lambda x: x[0])
        compressed = []
        for dt, typ in changes_sorted:
            if not compressed or compressed[-1][1] != typ:
                compressed.append((dt, typ))
        hospital_intervals[name] = compressed
    return hospital_intervals

def get_all_change_dates(hospital_intervals):
    s = set()
    for changes in hospital_intervals.values():
        for dt, typ in changes:
            s.add(pd.to_datetime(dt).to_pydatetime())
    return sorted(list(s))

# ---------------- Helpers: Gaza polygon normalization ----------------

def normalize_clip_shape(clip_input, hospitals_gdf=None, acled_gdf=None):
    """
    Accepts:
      - path to geojson/shapefile (string)
      - GeoDataFrame / GeoSeries
      - shapely geometry or GeoJSON dict
      - None (then use bbox around hospitals+attacks)
    Returns:
      GeoDataFrame (single row) in CRS_WGS84
    """
    if clip_input is None:
        # fallback to bbox around points
        if hospitals_gdf is None or acled_gdf is None:
            raise ValueError("clip_input is None and hospitals/acled not provided for bbox fallback.")
        combined = pd.concat([hospitals_gdf.dropna(subset=["geometry"]), acled_gdf.dropna(subset=["geometry"])], sort=False)
        if combined.empty:
            return gpd.GeoDataFrame(geometry=[box(34.2, 31.05, 35.6, 31.7)], crs=CRS_WGS84)
        minx, miny, maxx, maxy = combined.total_bounds
        bufx = (maxx - minx) * 0.1 + BUFFER_DEG
        bufy = (maxy - miny) * 0.1 + BUFFER_DEG
        return gpd.GeoDataFrame(geometry=[box(minx - bufx, miny - bufy, maxx + bufx, maxy + bufy)], crs=CRS_WGS84)

    # If string path
    if isinstance(clip_input, str):
        if not os.path.exists(clip_input):
            raise FileNotFoundError(f"GAZA_SHAPEFILE path not found: {clip_input}")
        g = gpd.read_file(clip_input)
        if g.crs is None:
            g = g.set_crs(CRS_WGS84)
        g = g.to_crs(CRS_WGS84)
        unified = g.dissolve(by=None).reset_index(drop=True)
        return unified

    if isinstance(clip_input, gpd.GeoDataFrame):
        g = clip_input.copy()
        if g.crs is None:
            g = g.set_crs(CRS_WGS84)
        g = g.to_crs(CRS_WGS84)
        return g.dissolve(by=None).reset_index(drop=True)

    if isinstance(clip_input, gpd.GeoSeries):
        return gpd.GeoDataFrame(geometry=[unary_union(clip_input.values)], crs=CRS_WGS84)

    if hasattr(clip_input, "geom_type"):
        return gpd.GeoDataFrame(geometry=[clip_input], crs=CRS_WGS84)

    if isinstance(clip_input, dict):
        t = clip_input.get("type", "").lower()
        geom = None
        if t == "featurecollection":
            geoms = []
            for f in clip_input.get("features", []):
                g = f.get("geometry", None)
                if g:
                    geoms.append(shape(g))
            if not geoms:
                raise ValueError("FeatureCollection contained no valid geometries.")
            geom = unary_union(geoms)
        elif t == "feature":
            geom = shape(clip_input.get("geometry"))
        else:
            features = clip_input.get("features", None)
            if features:
                geoms = [shape(f["geometry"]) for f in features if "geometry" in f]
                geom = unary_union(geoms)
        if geom is None:
            raise ValueError("Unsupported GeoJSON dict for clip_input.")
        return gpd.GeoDataFrame(geometry=[geom], crs=CRS_WGS84)

    raise TypeError("Unsupported clip_input type for normalize_clip_shape")

# ---------------- Safer Voronoi builder (preferred shapely, bounded SciPy fallback) ----------------

from shapely import geometry as shapely_geom
try:
    # shapely.ops.voronoi_diagram exists in Shapely >=2.0
    from shapely.ops import voronoi_diagram
    _HAS_SHAPELY_VORONOI = True
except Exception:
    voronoi_diagram = None
    _HAS_SHAPELY_VORONOI = False

def voronoi_polygons_clipped(points_gdf, clip_gdf):
    """
    Build Voronoi regions for points_gdf and clip them to clip_gdf (WGS84).
    Safer: prefers shapely.voronoi_diagram with a bounded envelope; falls back to SciPy Voronoi
    with a bbox in projected CRS to avoid infinite/unbounded polygons.
    Returns GeoDataFrame with ['geometry','Hospital'] in WGS84.
    """
    t0 = time.time()
    if points_gdf.empty:
        return gpd.GeoDataFrame(columns=["geometry", "Hospital"], crs=CRS_WGS84)

    npts = len(points_gdf)
    if npts > VORONOI_POINT_CAP:
        raise RuntimeError(f"Too many hospital points ({npts}) for Voronoi (cap={VORONOI_POINT_CAP}). Reduce points or increase cap.")

    # prepare projected and wgs geometries
    pts_wgs = points_gdf.reset_index(drop=True).to_crs(CRS_WGS84)
    pts_proj = points_gdf.reset_index(drop=True).to_crs(CRS_WEBMERC)

    # union clip polygon
    clip_union = clip_gdf.unary_union

    # Build a modest bounding envelope in projected coordinates for safe clipping
    clip_proj = gpd.GeoSeries([clip_union], crs=CRS_WGS84).to_crs(CRS_WEBMERC)
    minx, miny, maxx, maxy = clip_proj.total_bounds
    buf = VORONOI_BUFFER_METERS
    bbox_proj = shapely_geom.box(minx - buf, miny - buf, maxx + buf, maxy + buf)

    # Try shapely.voronoi_diagram if available
    if _HAS_SHAPELY_VORONOI:
        try:
            multip = shapely_geom.MultiPoint([(pt.x, pt.y) for pt in pts_proj.geometry])
            vor = voronoi_diagram(multip, envelope=bbox_proj, tolerance=0.0)
            # extract polygons from vor (which may be GeometryCollection)
            poly_list = []
            if vor.is_empty:
                poly_list = []
            else:
                # keep only polygonal pieces
                try:
                    for g in vor.geoms:
                        if isinstance(g, (Polygon, MultiPolygon)):
                            poly_list.append(g)
                except Exception:
                    # vor may be a Polygon directly
                    if isinstance(vor, (Polygon, MultiPolygon)):
                        poly_list = [vor]
            # assign polygons to nearest hospital by centroid
            polys_proj_gdf = gpd.GeoDataFrame(geometry=poly_list, crs=CRS_WEBMERC)
            polys_wgs = polys_proj_gdf.to_crs(CRS_WGS84).reset_index(drop=True)

            hosp_points_wgs = pts_wgs.set_geometry(pts_wgs.geometry).copy()
            assigned = []
            for poly in polys_wgs.geometry:
                if poly is None or poly.is_empty:
                    continue
                rep = poly.representative_point()
                dists = hosp_points_wgs.geometry.distance(rep)
                nearest_idx = int(dists.idxmin())
                hosp_name = hosp_points_wgs.loc[nearest_idx, 'Hospital']
                clipped_poly = poly.intersection(clip_union)
                if clipped_poly is None or clipped_poly.is_empty:
                    continue
                assigned.append((hosp_name, clipped_poly))

            # build result ensuring every hospital present
            rows = []
            for hosp in pts_wgs['Hospital'].values:
                polys_for = [p for (h, p) in assigned if h == hosp]
                geom_union = unary_union(polys_for) if polys_for else Polygon()
                rows.append({'Hospital': hosp, 'geometry': geom_union})
            result = gpd.GeoDataFrame(rows, crs=CRS_WGS84)
            elapsed = time.time() - t0
            print(f"[voronoi] shapely.voronoi_diagram used; time={elapsed:.2f}s, points={npts}")
            return result[['geometry','Hospital']]
        except Exception as e:
            print(f"[voronoi] shapely.voronoi_diagram failed: {e}. Falling back to SciPy bounded Voronoi.")

    # ---------- SciPy fallback (bounded by bbox_proj) ----------
    coords = np.array([(pt.x, pt.y) for pt in pts_proj.geometry])
    vor = Voronoi(coords)

    proj_polys = []
    for pt_idx, region_index in enumerate(vor.point_region):
        region = vor.regions[region_index]
        if not region or -1 in region:
            # infinite region: build convex hull of site + bbox corners (bounded)
            site = coords[pt_idx]
            bbox_coords = np.array(bbox_proj.exterior.coords)
            pts_for_hull = np.vstack([site, bbox_coords])
            hull = shapely_geom.MultiPoint([tuple(p) for p in pts_for_hull]).convex_hull
            poly = hull
        else:
            try:
                verts = [vor.vertices[i] for i in region]
                poly = Polygon(verts)
            except Exception:
                site = coords[pt_idx]
                bbox_coords = np.array(bbox_proj.exterior.coords)
                pts_for_hull = np.vstack([site, bbox_coords])
                hull = shapely_geom.MultiPoint([tuple(p) for p in pts_for_hull]).convex_hull
                poly = hull
        # clip to bbox_proj to keep finite
        poly_clipped = poly.intersection(bbox_proj)
        proj_polys.append(poly_clipped)

    polys_proj_gdf = gpd.GeoDataFrame(geometry=proj_polys, crs=CRS_WEBMERC)
    polys_wgs = polys_proj_gdf.to_crs(CRS_WGS84).reset_index(drop=True)

    # assign each polygon to the corresponding hospital (order preserved)
    hosp_names = pts_wgs['Hospital'].values
    rows = []
    for i, poly in enumerate(polys_wgs.geometry):
        try:
            clipped = poly.intersection(clip_union)
        except Exception:
            clipped = Polygon()
        rows.append({'Hospital': hosp_names[i], 'geometry': (clipped if clipped is not None else Polygon())})
    result = gpd.GeoDataFrame(rows, crs=CRS_WGS84)

    # If leftover area in clip_union remains, assign pieces to nearest hospital
    covered = unary_union([g for g in result.geometry if g is not None and not g.is_empty])
    leftover = clip_union.difference(covered) if covered is not None else clip_union
    if leftover and not leftover.is_empty:
        pieces = [leftover] if isinstance(leftover, Polygon) else list(leftover.geoms)
        hosp_pts = pts_wgs.geometry
        for piece in pieces:
            rep = piece.representative_point()
            dists = hosp_pts.distance(rep)
            nearest_idx = int(dists.idxmin())
            cur = result.loc[result['Hospital'] == hosp_names[nearest_idx], 'geometry'].values[0]
            if cur is None or cur.is_empty:
                result.loc[result['Hospital'] == hosp_names[nearest_idx], 'geometry'] = [piece]
            else:
                result.loc[result['Hospital'] == hosp_names[nearest_idx], 'geometry'] = [cur.union(piece)]

    elapsed = time.time() - t0
    print(f"[voronoi] SciPy fallback complete; time={elapsed:.2f}s, points={npts}")
    return result[['geometry','Hospital']]

# ---------------- Utilities ----------------

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate haversine distance between points in kilometers.
    All inputs are numpy arrays (can be 1D or 2D), function returns distances in km.
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371.0 * c
    return km

def find_closest_hospital_for_attacks(acled_gdf, hosp_gdf, hospital_intervals, date_col):
    """
    For each attack in acled_gdf, find the closest open hospital.
    Returns a DataFrame with columns: 'Closest Hospital', 'Distance to Closest Hospital (km)', 'In Proximity?'
    'Closest Hospital' is the closest hospital that was open at the time of the attack.
    'In Proximity?' is 'Yes' if the closest open hospital is within 5 km, 'No' otherwise.
    """
    result_data = []
    
    # Get hospital coordinates as arrays
    hosp_names = hosp_gdf['Hospital'].values
    hosp_lons = hosp_gdf.geometry.x.values
    hosp_lats = hosp_gdf.geometry.y.values
    
    # Get attack coordinates
    attack_lons = acled_gdf.geometry.x.values
    attack_lats = acled_gdf.geometry.y.values
    attack_dates = acled_gdf[date_col].values
    
    for i in range(len(acled_gdf)):
        attack_date = pd.to_datetime(attack_dates[i]).to_pydatetime()
        attack_lon = attack_lons[i]
        attack_lat = attack_lats[i]
        
        # Find which hospitals were open at the time of this attack
        open_hosp_indices = []
        open_hosp_names_list = []
        for j, hosp_name in enumerate(hosp_names):
            # Check if hospital was open at attack date
            if hosp_name in hospital_intervals:
                evs = hospital_intervals[hosp_name]
                last_status = None
                for dt, typ in evs:
                    if dt <= attack_date:
                        last_status = typ
                    else:
                        break
                if last_status == "Open":
                    open_hosp_indices.append(j)
                    open_hosp_names_list.append(hosp_name)
        
        if not open_hosp_indices:
            # No open hospitals at this time
            result_data.append({
                'Closest Hospital': None,
                'Distance to Closest Hospital (km)': np.nan,
                'In Proximity?': 'No'
            })
        else:
            # Calculate distances to all open hospitals
            open_hosp_lons = hosp_lons[open_hosp_indices]
            open_hosp_lats = hosp_lats[open_hosp_indices]
            
            # Calculate distances using haversine
            # Broadcast single attack point to all open hospitals
            distances = haversine_np(
                np.full(len(open_hosp_lons), attack_lon),
                np.full(len(open_hosp_lats), attack_lat),
                open_hosp_lons,
                open_hosp_lats
            )
            
            # Find closest
            closest_idx = np.argmin(distances)
            closest_dist_km = distances[closest_idx]
            closest_hosp_name = open_hosp_names_list[closest_idx]
            
            # Check if within 5 km
            in_proximity = 'Yes' if closest_dist_km <= 5.0 else 'No'
            
            result_data.append({
                'Closest Hospital': closest_hosp_name,
                'Distance to Closest Hospital (km)': closest_dist_km,
                'In Proximity?': in_proximity
            })
    
    return pd.DataFrame(result_data)

def pts_to_heatlist(acled_gdf, round_decimals=5):
    """
    Turn an ACLED GeoDataFrame into a HeatMap-style list [[lat, lon, weight], ...]
    Aggregates by rounded coordinates so co-located events increase weight.
    Returns (heat_list, max_weight) where max_weight is used as HeatMap max_val.
    """
    if acled_gdf is None or acled_gdf.empty:
        return [], 1

    df = acled_gdf.copy()
    # ensure geometry present
    df = df[~df.geometry.is_empty & df.geometry.notna()].copy()
    if df.empty:
        return [], 1

    df["lon"] = df.geometry.x
    df["lat"] = df.geometry.y
    # round to avoid tiny floating differences for truly colocated events
    df["lon_r"] = df["lon"].round(round_decimals)
    df["lat_r"] = df["lat"].round(round_decimals)

    grouped = df.groupby(["lat_r", "lon_r"]).size().reset_index(name="count")
    # build heat list in the format expected by folium.plugins.HeatMap: [lat, lon, weight]
    heat = grouped.apply(lambda r: [float(r["lat_r"]), float(r["lon_r"]), float(r["count"])], axis=1).tolist()
    max_weight = float(grouped["count"].max()) if not grouped.empty else 1.0
    return heat, max_weight


# ---------------- Main pipeline ----------------
def main():
    start_run = time.time()
    run_dt = datetime.now()
    run_display = run_dt.strftime("%d/%m/%Y %H:%M:%S")
    run_tag = run_dt.strftime("%Y%m%d_%H%M%S")
    print("=" * 80)
    print(f"CODE EXECUTION STARTED")
    print(f"Date and Time: {run_display}")
    print(f"Run Tag: {run_tag}")
    print("=" * 80)

    print("Reading hospitals...")
    hospitals_df, schedule_meta = read_hospitals_table(HOSP_PATH)
    print(f"Loaded {len(hospitals_df)} hospitals; schedule events detected: {len(schedule_meta)}")

    hospital_intervals = build_hospital_open_intervals(hospitals_df, schedule_meta)
    all_change_dates = get_all_change_dates(hospital_intervals)
    print(f"Unique change dates across hospitals: {len(all_change_dates)}")

    print("Reading ACLED file...")
    acled = pd.read_excel(ACLED_PATH)
    ac_cols = list(acled.columns)

    # prefer explicit 'event_date' column if present
    date_col = None
    if any(str(c).strip().lower() == "event_date" for c in ac_cols):
        # find the exact column name that matches event_date
        date_col = next(c for c in ac_cols if str(c).strip().lower() == "event_date")
    else:
        # fallback detection
        date_col = next((c for c in ac_cols if "date" in str(c).lower() or "event_date" in str(c).lower()), None)
        if date_col is None:
            for c in ac_cols:
                try:
                    pd.to_datetime(acled[c].dropna().iloc[0])
                    date_col = c
                    break
                except Exception:
                    continue
    if date_col is None:
        raise ValueError("Couldn't detect date column in ACLED file. Looked for 'event_date' and other date-like columns.")

    lon_col = next((c for c in ac_cols if "lon" in str(c).lower() or "longitude" in str(c).lower() or str(c).lower() == "x"), None)
    lat_col = next((c for c in ac_cols if "lat" in str(c).lower() or "latitude" in str(c).lower() or str(c).lower() == "y"), None)
    if lon_col is None or lat_col is None:
        raise ValueError(f"Couldn't detect lon/lat columns in ACLED file. Columns: {ac_cols}")

    # parse event_date explicitly and drop bad rows
    acled[date_col] = pd.to_datetime(acled[date_col], errors="coerce")
    acled = acled.dropna(subset=[lon_col, lat_col, date_col]).copy()

    # Create GeoDataFrame using the precise lat/lon columns detected
    acled_gdf = gpd.GeoDataFrame(
        acled,
        geometry=gpd.points_from_xy(acled[lon_col].astype(float), acled[lat_col].astype(float)),
        crs=CRS_WGS84
    )

    # hospitals geo
    hosp_gdf = gpd.GeoDataFrame(
        hospitals_df.dropna(subset=["lon", "lat"]),
        geometry=gpd.points_from_xy(hospitals_df["lon"].astype(float), hospitals_df["lat"].astype(float)),
        crs=CRS_WGS84
    )
    print(f"Hospitals with valid coords: {len(hosp_gdf)}")

    # Calculate closest hospital for each attack and add columns
    print("Calculating closest hospitals for each attack...")
    closest_hosp_df = find_closest_hospital_for_attacks(acled_gdf, hosp_gdf, hospital_intervals, date_col)
    
    # Add the new columns to acled (convert back to DataFrame first, then add columns)
    acled_with_cols = acled_gdf.copy()
    acled_with_cols['Closest Hospital'] = closest_hosp_df['Closest Hospital'].values
    acled_with_cols['Distance to Closest Hospital (km)'] = closest_hosp_df['Distance to Closest Hospital (km)'].values
    acled_with_cols['In Proximity?'] = closest_hosp_df['In Proximity?'].values
    
    # Filter to only include attacks within 5km of closest hospital
    acled_within_5km = acled_with_cols[acled_with_cols['In Proximity?'] == 'Yes'].copy()
    print(f"Total attacks: {len(acled_with_cols)}, Attacks within 5km: {len(acled_within_5km)}")
    
    # Save the modified ACLED file - only attacks within 5km (avoid overwriting prior outputs)
    output_acled_path = os.path.join(OUT_DIR, f"v2_ACLED_within_5km_{run_tag}.xlsx")
    # Convert GeoDataFrame back to DataFrame for Excel export (drop geometry column)
    acled_output = acled_within_5km.drop(columns=['geometry']).copy()
    acled_output.to_excel(output_acled_path, index=False)
    print(f"Saved modified ACLED file (only attacks within 5km) with hospital proximity columns to: {output_acled_path}")
    
    # normalize clip (Gaza)
    clip_input = GAZA_SHAPEFILE if (GAZA_SHAPEFILE and os.path.exists(GAZA_SHAPEFILE)) else None
    if clip_input:
        print("Using Gaza polygon:", clip_input)
    else:
        print("No valid Gaza polygon provided; will fallback to bbox around data.")

    try:
        clip_gdf = normalize_clip_shape(clip_input, hospitals_gdf=hosp_gdf, acled_gdf=acled_gdf)
    except Exception as e:
        print("normalize_clip_shape failed:", e)
        # fallback bbox
        combined = pd.concat([hosp_gdf.geometry, acled_gdf.geometry])
        minx, miny, maxx, maxy = combined.total_bounds
        clip_gdf = gpd.GeoDataFrame(geometry=[box(minx - 0.05, miny - 0.05, maxx + 0.05, maxy + 0.05)], crs=CRS_WGS84)
        print("Using fallback bbox:", clip_gdf.total_bounds)

    # For each window, build intervals and maps
    maps_created = 0

    # helper: precise aggregation for heatmap (uses event_date filtering already done below)
    def pts_to_heatlist_precise(acled_subset_gdf, round_decimals=6):
        """
        Aggregate attack points by rounded lat/lon to produce weighted heatmap points.
        Uses a relatively high precision (default 6 decimals) to remain specific.
        Returns (heat_list, max_weight).
        """
        if acled_subset_gdf is None or acled_subset_gdf.empty:
            return [], 1.0

        df = acled_subset_gdf.copy()
        # remove empties
        df = df[~df.geometry.is_empty & df.geometry.notna()].copy()
        if df.empty:
            return [], 1.0

        # extract precise coords from geometry (use geometry.x / geometry.y directly)
        df["lon"] = df.geometry.x.astype(float)
        df["lat"] = df.geometry.y.astype(float)

        # round coordinates to given decimals to aggregate near-identical points while preserving precision
        df["lon_r"] = df["lon"].round(round_decimals)
        df["lat_r"] = df["lat"].round(round_decimals)

        # group by rounded coordinates to count number of attack records at each precise location
        grouped = df.groupby(["lat_r", "lon_r"]).size().reset_index(name="count")

        # convert to HeatMap format: [lat, lon, weight]
        heat = grouped.apply(lambda r: [float(r["lat_r"]), float(r["lon_r"]), float(r["count"])], axis=1).tolist()
        max_weight = float(grouped["count"].max()) if not grouped.empty else 1.0
        return heat, max_weight

    for window_start, window_end in WINDOWS:
        dates_in_window = [d for d in all_change_dates if (d >= window_start and d <= window_end)]
        combined_dates = sorted(list(set(dates_in_window + [window_start, window_end])))
        for a, b in zip(combined_dates[:-1], combined_dates[1:]):
            # We will count attacks whose event_date is in [a, b)
            interval_mid = a + (b - a) / 2

            # find open hospitals at mid
            open_hosp_names = []
            for hosp, evs in hospital_intervals.items():
                last_status = None
                for dt, typ in evs:
                    if dt <= interval_mid:
                        last_status = typ
                    else:
                        break
                if last_status == "Open":
                    open_hosp_names.append(hosp)
            if not open_hosp_names:
                print(f"[{a.date()} to {b.date()}] No open hospitals -> skipping.")
                continue

            open_pts = hosp_gdf[hosp_gdf["Hospital"].isin(open_hosp_names)].copy()
            if open_pts.empty:
                print(f"[{a.date()} to {b.date()}] Open hospital names found but no coordinates -> skipping.")
                continue

            # filter ACLED by event_date explicitly - only use attacks within 5km (already filtered)
            attacks_interval_filtered = acled_within_5km[(acled_within_5km[date_col] >= a) & (acled_within_5km[date_col] < b)].copy()
            print(f"[interval {a.date()}->{b.date()}] Attacks within 5km: {len(attacks_interval_filtered)}")

            # Build voronoi polygons clipped to Gaza
            t_v0 = time.time()
            vor_polys = voronoi_polygons_clipped(open_pts, clip_gdf)
            t_v1 = time.time()
            print(f"[interval {a.date()}->{b.date()}] Voronoi build time: {t_v1 - t_v0:.2f}s")

            # Build folium map
            minx, miny, maxx, maxy = clip_gdf.total_bounds
            center = [(miny + maxy) / 2, (minx + maxx) / 2]
            m = folium.Map(location=center, zoom_start=11)

            # color palette (used for outline strokes)
            palette = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#ffff33", "#a65628", "#f781bf", "#999999"]
            random.shuffle(palette)

            # add polygons as outlines only (no fill)
            for i, row in vor_polys.reset_index(drop=True).iterrows():
                hosp_name = row['Hospital']
                geom = row['geometry']
                if geom is None or geom.is_empty:
                    continue
                style = {
                    "fillOpacity": 0.0,     # no fill
                    "fill": False,
                    "color": palette[i % len(palette)],   # stroke color
                    "weight": 2,            # stroke width
                    "opacity": 0.9
                }
                folium.GeoJson(
                    data=gpd.GeoSeries([geom]).__geo_interface__,
                    name=hosp_name,
                    tooltip=str(hosp_name),
                    style_function=lambda feature, style=style: style
                ).add_to(m)

            # add hospital markers (outline-only small circle)
            for _, hr in open_pts.iterrows():
                folium.CircleMarker(
                    location=(hr.geometry.y, hr.geometry.x),
                    radius=4,
                    popup=str(hr.Hospital),
                    fill=True,
                    fillOpacity=0.9,
                    weight=1
                ).add_to(m)

            # add weighted heatmap of attacks in interval (if any). Use high precision rounding for lat/lon.
            # Only show attacks within 5 km of closest hospital
            if not attacks_interval_filtered.empty:
                heat, max_weight = pts_to_heatlist_precise(attacks_interval_filtered, round_decimals=6)

                # gradient mapping (0..1)
                gradient = {
                    0.0: 'blue',
                    0.2: 'lime',
                    0.4: 'yellow',
                    0.6: 'orange',
                    0.8: 'red',
                    1.0: 'darkred'
                }

                HeatMap(
                    heat,
                    radius=12,
                    blur=18,
                    max_zoom=13,
                    gradient=gradient,
                    max_val=max_weight
                ).add_to(m)

                # Add legend/colorbar matching gradient & show counts based on event_date aggregation
                legend_html = f"""
                <div style="
                    position: fixed;
                    bottom: 30px;
                    left: 10px;
                    z-index:9999;
                    background:white;
                    padding:8px;
                    border:1px solid grey;
                    font-size:12px;
                    ">
                  <b>Attack density (per interval)</b><br/>
                  <div style="width:200px; height:12px; background:linear-gradient(to right,
                        {gradient[0.0]}, {gradient[0.2]}, {gradient[0.4]}, {gradient[0.6]}, {gradient[0.8]}, {gradient[1.0]});
                        margin-top:6px; border:1px solid #ccc;"></div>
                  <div style="display:flex; justify-content:space-between; margin-top:4px;">
                    <span>0</span><span>{int(max_weight//2)}</span><span>{int(max_weight)}</span>
                  </div>
                  <div style="margin-top:6px;">
                    <small>Counts reflect number of ACLED records (by <b>event_date</b>) aggregated at precise coordinates (rounded to 6 decimals).</small>
                  </div>
                </div>
                """
                m.get_root().html.add_child(folium.Element(legend_html))

            # add title box
            title_html = f"""
            <div style="position: fixed; top: 10px; left: 10px; z-index: 9999; background: white;
                         padding: 8px; border: 1px solid grey; font-size:12px;">
                <b>Run:</b> {run_display}<br/>
                <b>Interval:</b> {a.strftime('%Y-%m-%d')} → {b.strftime('%Y-%m-%d')}<br/>
                <b>Open hospitals:</b> {', '.join(open_hosp_names)}
            </div>
            """
            m.get_root().html.add_child(folium.Element(title_html))

            fname = os.path.join(OUT_DIR, f"v2_gaza_catchment_{a.strftime('%Y%m%d')}_to_{b.strftime('%Y%m%d')}_{run_tag}.html")
            m.save(fname)
            maps_created += 1
            print(f"Saved map: {fname}")

    total_elapsed = time.time() - start_run
    end_dt = datetime.now()
    end_display = end_dt.strftime("%d/%m/%Y %H:%M:%S")
    print("=" * 80)
    print(f"CODE EXECUTION COMPLETED")
    print(f"Completion Date and Time: {end_display}")
    print(f"Total execution time: {total_elapsed:.2f} seconds ({total_elapsed/60:.2f} minutes)")
    print(f"Maps created: {maps_created}")
    print("=" * 80)


if __name__ == "__main__":
    main()


CODE EXECUTION STARTED
Date and Time: 02/02/2026 14:13:06
Run Tag: 20260202_141306
Reading hospitals...
Loaded 5 hospitals; schedule events detected: 5
Unique change dates across hospitals: 5
Reading ACLED file...
Hospitals with valid coords: 5
Calculating closest hospitals for each attack...
Total attacks: 28080, Attacks within 5km: 12943
Saved modified ACLED file (only attacks within 5km) with hospital proximity columns to: ./v2_ACLED_within_5km_20260202_141306.xlsx
Using Gaza polygon: gaza_boundary.geojson
[interval 2023-12-01->2024-03-18] Attacks within 5km: 3098
[voronoi] shapely.voronoi_diagram used; time=13.46s, points=5
[interval 2023-12-01->2024-03-18] Voronoi build time: 13.46s
Saved map: ./v2_gaza_catchment_20231201_to_20240318_20260202_141306.html
[interval 2024-03-18->2024-04-01] Attacks within 5km: 337
[voronoi] shapely.voronoi_diagram used; time=0.56s, points=4
[interval 2024-03-18->2024-04-01] Voronoi build time: 0.56s
Saved map: ./v2_gaza_catchment_20240318_to_20240401